# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [42]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [43]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/sgupta/Desktop/Udacity/Data Engineer/Project2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [44]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        

In [45]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [46]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)
    
# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [47]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )

except Exception as e:
    print(e)

#### Set Keyspace

In [48]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [49]:
# Create table for Query 1
query1 = "CREATE TABLE IF NOT EXISTS songs_in_user_session "
query1 = query1 + "(sessionId int, itemInSession int, artist text, songTitle text, songLength float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)
    
# Create table for Query 2
query2 = "CREATE TABLE IF NOT EXISTS session_info_per_user "
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, songTitle text, userName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
    
# Create table for Query 3
query3 = "CREATE TABLE IF NOT EXISTS song_analytics_table "
query3 = query3 + "(songTitle text, userId int, userName text, PRIMARY KEY (songTitle, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [50]:
# Populate the three tables using the data from csv files
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Columns used in Queries
        sessionId = int(line[8])
        itemInSession = int(line[3])
        artist = line[0]
        songTitle = line[9]
        songLength = float(line[5])
        userId =  int(line[10])
        userName = line[1] + " " + line[4]
        
        query1 = "INSERT INTO songs_in_user_session (sessionId, itemInSession, artist, songTitle, songLength) "
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (sessionId, itemInSession, artist, songTitle, songLength))
        
        query2 = "INSERT INTO session_info_per_user (userId, sessionId, itemInSession, artist, songTitle, userName) "
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query2, (userId, sessionId, itemInSession, artist, songTitle, userName))
        
        query3 = "INSERT INTO song_analytics_table (songTitle, userId, userName) "
        query3 = query3 + "VALUES (%s, %s, %s)"
        session.execute(query3, (songTitle, userId, userName))

#### Do a SELECT to verify that the data have been inserted into each table

In [51]:
## Add in the SELECT statement to verify the data was entered into the table

# Test Select for Query 1
select_query_1 = "SELECT * FROM songs_in_user_session "
select_query_1 = select_query_1 + "WHERE sessionId=139"
try:
    results = session.execute(select_query_1)
except Exception as e:
    print(e)
    
for row in results:
    print(row)
    break

# Test Select for Query 2
select_query_2 = "SELECT * FROM session_info_per_user "
select_query_2 = select_query_2 + "WHERE userId=8 "
select_query_2 = select_query_2 + "ORDER BY itemInSession"

try:
    results = session.execute(select_query_2)
except Exception as e:
    print(e)
    
for row in results:
    print(row)
    break

# Test Select for Query 3
select_query_3 = "SELECT * FROM song_analytics_table "
select_query_3 = select_query_3 + "WHERE songTitle='Flat 55'"
try:
    results = session.execute(select_query_3)
except Exception as e:
    print(e)
    
for row in results:
    print(row)
    break

Row(sessionid=139, iteminsession=1, artist="Des'ree", songlength=246.30812072753906, songtitle='You Gotta Be')
Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"
Row(sessionid=139, iteminsession=1, artist="Des'ree", songlength=246.30812072753906, songtitle='You Gotta Be')
Row(songtitle='Flat 55', userid=8, username='Kaylee Summers')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [52]:
## Query 1: Give me the artist, song title and song's length in the music app history\
## that was heard during  sessionId = 338, and itemInSession  = 4

query_1 = "SELECT artist, songTitle, songLength FROM songs_in_user_session "
query_1 = query_1 + "WHERE sessionId=338 AND itemInSession=4"
try:
    results = session.execute(query_1)
except Exception as e:
    print(e)
    
for row in results:
    print(row)

Row(artist='Faithless', songtitle='Music Matters (Mark Knight Dub)', songlength=495.30731201171875)


In [53]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query_2 = "SELECT artist, songTitle, userName FROM session_info_per_user "
query_2 = query_2 + "WHERE userId=10 AND sessionId=182 "
query_2 = query_2 + "ORDER BY itemInSession"
try:
    results = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in results:
    print(row)      

Row(artist='Down To The Bone', songtitle="Keep On Keepin' On", username='Sylvie Cruz')
Row(artist='Three Drives', songtitle='Greece 2000', username='Sylvie Cruz')
Row(artist='Sebastien Tellier', songtitle='Kilometer', username='Sylvie Cruz')
Row(artist='Lonnie Gordon', songtitle='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', username='Sylvie Cruz')


In [54]:
## Query 3: Give me every user name (first and last) in my music app history\
## who listened to the song 'All Hands Against His Own'

query_3 = "SELECT userName FROM song_analytics_table "
query_3 = query_3 + "WHERE songTitle='All Hands Against His Own'"
try:
    results = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in results:
    print(row)      

Row(username='Jacqueline Lynch')
Row(username='Tegan Levine')
Row(username='Sara Johnson')


### Drop the tables before closing out the sessions

In [55]:
## Drop the table before closing out the sessions
drop_query_1 = "DROP TABLE IF EXISTS songs_in_user_session"
try:
    results = session.execute(drop_query_1)
except Exception as e:
    print(e)
    
drop_query_2 = "DROP TABLE IF EXISTS session_info_per_user"
try:
    results = session.execute(drop_query_2)
except Exception as e:
    print(e)
    
drop_query_3 = "DROP TABLE IF EXISTS song_analytics_table"
try:
    results = session.execute(drop_query_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [56]:
session.shutdown()
cluster.shutdown()